In [2]:
import pandas as pd
import numpy as np
import json

import folium

In [3]:
# 나의 루트 경로 설정
path_dateset = 'G:/내 드라이브/DataSet/'

## 데이터 불러오기

### 자전거 도로 정보

In [8]:
geo_path = path_dateset + '_최종 병합 파일\\서울시 자전거 도로 데이터\\' + 'ddareng_road.geojson'
# geo_str = json.load(open(geo_path, encoding='cp949'))
# encoding을 UTF-8로 지정
with open(geo_path, 'rt', encoding='utf-8') as f:
    geo_str = json.load(f)
# # ============================

### 자치구 구분 정보

In [11]:
geo_path = path_dateset + 'seoul_municipalities_geo_simple.json'
# geo_str = json.load(open(geo_path, encoding='cp949'))
# encoding을 UTF-8로 지정
with open(geo_path, 'rt', encoding='utf-8') as f:
    geo_str2 = json.load(f)
# # ============================

### 자전거 대여소 정보

In [14]:
cols = ['대여소번호', '보관소(대여소)명', '자치구', '상세주소', '위도', '경도', '설치시기', '거치대수(LCD)', '거치대수(QR)', '운영방식']
# rental = pd.read_csv(path_dateset + '서울시 공공자전거 대여소 정보/공공자전거 대여소 정보(24.6월 기준).xlsx', encoding = 'utf-8', names = cols, index_col = False)
df_bike_rentalstation = pd.read_excel(path_dateset + '_최종 병합 파일\서울시 공공자전거 대여소 정보/공공자전거 대여소 정보(24.6월 기준).xlsx', sheet_name='대여소현황', names = cols)
df_bike_rentalstation = df_bike_rentalstation.iloc[4:]
df_bike_rentalstation.reset_index(drop = True, inplace = True)

In [18]:
df_bike_rentalstation.head(3)

,대여소번호,보관소(대여소)명,자치구,상세주소,위도,경도,설치시기,거치대수(LCD),거치대수(QR),운영방식
0,301.0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130 경복궁역 7번출구 앞,37.575794,126.971451,2015-10-07 12:03:46,20,20,QR
1,302.0,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130 경복궁역 4번출구 뒤,37.575947,126.97406,2015-10-07 12:04:22,12,12,QR
2,303.0,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189 세종로공원,37.57177,126.974663,2015-10-07 00:00:00,8,8,QR


In [20]:
df_bike_rentalstation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2763 entries, 0 to 2762
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   대여소번호      2763 non-null   float64       
 1   보관소(대여소)명  2763 non-null   object        
 2   자치구        2763 non-null   object        
 3   상세주소       2763 non-null   object        
 4   위도         2763 non-null   object        
 5   경도         2763 non-null   object        
 6   설치시기       2763 non-null   datetime64[ns]
 7   거치대수(LCD)  1444 non-null   object        
 8   거치대수(QR)   1662 non-null   object        
 9   운영방식       2763 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 216.0+ KB


---
# 자치구별 자전거 밀도확인

In [23]:
have_bikes = df_bike_rentalstation.groupby('자치구').size()

### 지하철역통합_20241031.parquet

In [28]:
filename = '_최종 병합 파일\\서울교통공사 역주소 및 전화번호\\지하철역통합_20241031.parquet'
df_train_station = pd.read_parquet(path_dateset + filename)
df_train_station.head(3)

,연번,역번호,호선,역명,역전화번호,도로명주소,지번주소,station_code,seoulmetro_code,external_code,station_name(kor),lat,lng
0,NaN,None,NaN,<NA>,<NA>,<NA>,<NA>,651.0,4703,943,419민주묘지,37.648235,127.008614
1,NaN,None,NaN,<NA>,<NA>,<NA>,<NA>,335.0,1907,109,가능,37.748577,127.044213
2,92.0,340,3.0,가락시장,02-6110-3501,서울특별시 송파구 송파대로 지하257(가락동),서울특별시 송파구 가락동 184-23 가락시장역(3호선),78.0,340,350,가락시장,37.492522,127.118234


In [30]:
df_train_station.columns

Index(['연번', '역번호', '호선', '역명', '역전화번호', '도로명주소', '지번주소', 'station_code',
       ' seoulmetro_code', ' external_code', ' station_name(kor)', 'lat',
       'lng'],
      dtype='object')

In [32]:
print(f'df_bike_rentalstation.shape[0] : {df_bike_rentalstation.shape[0]}')
print(f'df_train_station.shape[0] : {df_train_station.shape[0]}')

df_bike_rentalstation.shape[0] : 2763
df_train_station.shape[0] : 614


#### # Haversine 공식을 사용하여 두 지점 간의 거리 계산 함수
- 좌표 값만으로 거리를 구할 수 있다.

In [35]:
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371e3  # 지구 반지름 (미터 단위)
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c

In [37]:
# 자전거 대여소와 지하철역 간의 거리 확인
result = []
distance_limit = 100

# 자전거대여소 개수만큼 반복
for _, bike_station in df_bike_rentalstation.iterrows():
    bike_location = (bike_station['위도'], bike_station['경도'])
    nearby_stations_code = []    
    nearby_stations_name = []
    nearby_stations_distance = []
    nearby_stations_lat = []    # 위도
    nearby_stations_lng = []    # 경도

    # 지하철역 개수만큼 반복
    for _, train_station in df_train_station.iterrows():
        train_location = (train_station['lat'], train_station['lng'])
        distance = calculate_distance(bike_location[0], bike_location[1], train_location[0], train_location[1])

        # 지하철역과의 거리가 100m이하리면, 역정보 저장
        if distance <= distance_limit: # 거리 조건: 100m 이내
            nearby_stations_code.append(train_station[' station_name(kor)'])
            nearby_stations_name.append(train_station[' seoulmetro_code'])
            nearby_stations_distance.append(distance)
            nearby_stations_lat.append(train_station['lat'])
            nearby_stations_lng.append(train_station['lng'])
    
    # result.append({'대여소명': bike_station['보관소(대여소)명'], '근처 지하철역': nearby_stations})
    result.append({'대여소_번호': bike_station['대여소번호'],
                   '대여소_명': bike_station['보관소(대여소)명'],
                   '대여소_위도': bike_station['위도'],
                   '대여소_경도': bike_station['경도'],
                   '근처지하철역_번호': nearby_stations_code,
                   '근처지하철역_명': nearby_stations_name,
                   '근처지하철역_거리': nearby_stations_distance,
                   '근처지하철역_위도': nearby_stations_lat,
                   '근처지하철역_경도': nearby_stations_lng,
                  })

# 결과 출력
result_df = pd.DataFrame(result)
print(result_df)

      대여소_번호         대여소_명     대여소_위도      대여소_경도 근처지하철역_번호 근처지하철역_명  \
0      301.0   경복궁역 7번출구 앞  37.575794  126.971451        []       []   
1      302.0   경복궁역 4번출구 뒤  37.575947  126.974060     [경복궁]    [317]   
2      303.0   광화문역 1번출구 앞  37.571770  126.974663        []       []   
3      305.0        종로구청 옆  37.572559  126.978333        []       []   
4      307.0     서울역사박물관 앞  37.570000  126.971100        []       []   
...      ...           ...        ...         ...       ...      ...   
2758  5872.0       서경팰리스 앞  37.505764  126.908661        []       []   
2759  6057.0   을지로5가(방산시장)  37.567398  127.002014        []       []   
2760  6058.0   서울도시건축전시관 옆  37.566540  126.976845        []       []   
2761  6059.0         덕수중학교  37.564392  126.997299        []       []   
2762  6173.0   서울자동차운전전문학원  37.546135  126.821060        []       []   

                근처지하철역_거리    근처지하철역_위도    근처지하철역_경도  
0                      []           []           []  
1     [51.03307857106119]  

In [39]:
result_df.columns

Index(['대여소_번호', '대여소_명', '대여소_위도', '대여소_경도', '근처지하철역_번호', '근처지하철역_명',
       '근처지하철역_거리', '근처지하철역_위도', '근처지하철역_경도'],
      dtype='object')

In [53]:
result_df.shape[0]

2763

In [61]:
# '근처 지하철역' 컬럼에 데이터가 있는지 확인 후 필터링
df_train_O = result_df[result_df['근처지하철역_번호'].str.len() > 0]   # 지하철근처 대여소
df_train_X = result_df[result_df['근처지하철역_번호'].str.len() == 0]  # 지하철근처 아닌 대여소

In [63]:
print(df_train_O.shape[0])
print(df_train_X.shape[0])

315
2448


In [65]:
# 근처 지하철역이 2개 이상인 대여소 확인
for i, row in df_train_O.iterrows():
    if len(row['근처지하철역_번호']) > 1:
        print(row)

대여소_번호                                       2032.0
대여소_명                                    이수역 11번출구쪽
대여소_위도                                    37.485508
대여소_경도                                   126.981621
근처지하철역_번호                               [이수, 총신대입구]
근처지하철역_명                                [2738, 432]
근처지하철역_거리    [90.02237319355677, 90.02237319355677]
근처지하철역_위도                    [37.486263, 37.486263]
근처지하철역_경도                  [126.981989, 126.981989]
Name: 1951, dtype: object
대여소_번호                                       2230.0
대여소_명                                     이수역 4번 출구
대여소_위도                                    37.485828
대여소_경도                                   126.982628
근처지하철역_번호                               [이수, 총신대입구]
근처지하철역_명                                [2738, 432]
근처지하철역_거리    [74.24872003343553, 74.24872003343553]
근처지하철역_위도                    [37.486263, 37.486263]
근처지하철역_경도                  [126.981989, 126.981989]
Name: 2109, dtype: object


In [67]:
# 지도 생성
center = [37.541, 126.986]
# center on Liberty Bell, add marker
bike_map = folium.Map(location=center, zoom_start=13)
# 스타일 지정 예시 (선의 색상, 두께 등)
style = {
    'color': 'red',
    'weight': 3,
    'opacity': 0.7,
}

fg0 = folium.FeatureGroup(name="자전거_도로", show=False).add_to(bike_map)
# 자전거 도로 표시
# GeoJSON 파일을 불러와 스타일 적용
folium.GeoJson(
    geo_str,
    name='bike_road',
    style_function=lambda x: style
).add_to(fg0)

fg1 = folium.FeatureGroup(name="자전거_대여소(지하철O)", show=False).add_to(bike_map)
# 자전거 대여소 표시 (지하철 인근 O)
for i, row in df_train_O.iterrows():
    folium.Circle(
      location = [row['대여소_위도'], row['대여소_경도']],
      radius = 100,
      color='black',                  # 외곽선 색상
      fill=False,                     # 채우기 여부
      fill_color='blue',             # 채우기 색상
      fill_opacity=0.6,              # 채우기 불투명도
      popup=row['대여소_번호'],             # 클릭 시 표시되는 팝업
      tooltip=row['대여소_명'],                 # 마우스 오버 시 표시되는 툴팁
      #icon=folium.Icon(icon = 'bookmark', color = 'blue')
  ).add_to(fg1)

fg2 = folium.FeatureGroup(name="자전거_대여소(지하철X)", show=False).add_to(bike_map)
# 자전거 대여소 표시 (지하철 인근 X)
for i, row in df_train_X.iterrows():
    folium.Circle(
      location = [row['대여소_위도'], row['대여소_경도']],
      radius = 100,
      color='black',                  # 외곽선 색상
      fill=False,                     # 채우기 여부
      fill_color='green',             # 채우기 색상
      fill_opacity=0.6,              # 채우기 불투명도
      popup=row['대여소_번호'],             # 클릭 시 표시되는 팝업
      tooltip=row['대여소_명'],                 # 마우스 오버 시 표시되는 툴팁
      #icon=folium.Icon(icon = 'bookmark', color = 'blue')
  ).add_to(fg2)
    

fg3 = folium.FeatureGroup(name="지하철역", show=False).add_to(bike_map)
# 지하철역 표시
for i, row in filtered_df.iterrows():
  folium.CircleMarker(
      location = [row['근처지하철역_위도'][0], row['근처지하철역_경도'][0]],
      radius=2,                     # 반지름 (픽셀 단위)
      color='yellow',                  # 외곽선 색상
      fill=True,                     # 채우기 여부
      fill_color='yellow',             # 채우기 색상
      fill_opacity=0.6,              # 채우기 불투명도
      popup=row['근처지하철역_번호'][0],             # 클릭 시 표시되는 팝업
      tooltip=row['근처지하철역_명'][0]                 # 마우스 오버 시 표시되는 툴팁      
  ).add_to(fg3)

# 자치구 구분 표
geo_str2 = json.load(open('G:\\내 드라이브\\DataSet\\seoul_municipalities_geo_simple.json', encoding='utf-8'))
choropleth = folium.Choropleth(geo_data = geo_str2,
                  data = have_bikes,
                  name = 'gu',
                  key_on = 'feature.properties.SIG_KOR_NM',
                  fill_color = 'Blues')
# 1. GeoJsonTooltip을 사용하는 방법:
# folium.GeoJsonTooltip(fields=['SIG_ENG_NM'],
#                       aliases=['지역:'],
#                       style=('background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;')
# ).add_to(choropleth.geojson)
# 2. GeoJsonPopup을 사용하는 방법:
folium.GeoJsonPopup(fields=['SIG_KOR_NM'],
                    aliases=['지역:'],
                    style='font-weight: bold;'
).add_to(choropleth.geojson)
choropleth.add_to(bike_map)


folium.LayerControl().add_to(bike_map)

# jupyter 에서 곧장 지도 표시하기
#bike_map

# web에서 표시하기
# 지도 저장
bike_map.save("folium_bike_train_near.html")
import webbrowser
webbrowser.open("folium_bike_train_near.html")

True